In [0]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly
import plotly.graph_objects as go

In [0]:
filename = "^JKSE (3).csv"
df = pd.read_csv(filename)
print(df.info())

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 7 columns):
Date         1213 non-null object
Open         1213 non-null float64
High         1213 non-null float64
Low          1213 non-null float64
Close        1213 non-null float64
Adj Close    1213 non-null float64
Volume       1213 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 66.5+ KB
None


,Date,Close,Adj Close
Date,,,
2014-11-25,2014-11-25,5118.944824,5118.792969
2014-11-26,2014-11-26,5133.036133,5132.883789
2014-11-27,2014-11-27,5145.314941,5145.162109
2014-11-28,2014-11-28,5149.888184,5149.735352
2014-12-01,2014-12-01,5164.288086,5164.134766
2014-12-02,2014-12-02,5175.792969,5175.639160
2014-12-03,2014-12-03,5166.043945,5165.890625
2014-12-04,2014-12-04,5177.160156,5177.006348
2014-12-05,2014-12-05,5187.994141,5187.840332


In [0]:
trace = go.Scatter(
    x = df['Date'],
    y = df['Close'],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close (Rupiah)"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [0]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

970
243


In [0]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [0]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
48/48 [==============================] - 3s 61ms/step - loss: 19699997.8784
Epoch 2/25
48/48 [==============================] - 1s 10ms/step - loss: 13764974.3864
Epoch 3/25
48/48 [==============================] - 1s 12ms/step - loss: 4018993.0301
Epoch 4/25
48/48 [==============================] - 1s 11ms/step - loss: 1372888.9725
Epoch 5/25
48/48 [==============================] - 0s 10ms/step - loss: 833113.6780
Epoch 6/25
48/48 [==============================] - 0s 10ms/step - loss: 906720.2576
Epoch 7/25
48/48 [==============================] - 0s 10ms/step - loss: 461562.9471
Epoch 8/25
48/48 [==============================] - 0s 10ms/step - loss: 227768.1717
Epoch 9/25
48/48 [==============================] - 0s 10ms/step - loss: 123979.9123
Epoch 10/25
48/48 [==============================] - 1s 12ms/step - loss: 69204.5147
Epoch 11/25
48/48 [==============================] - 1s 12ms/step - loss: 39517.5153
Epoch 12/25
48/48 [==============================] - 0s 10m

In [0]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "IHSG",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [0]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 10
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [0]:
trace1 = go.Scatter(
    x = df['Date'].tolist(),
    y = close_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "IHSG",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()